<a href="https://colab.research.google.com/github/Muhamadsulton/kaggle/blob/main/nps_sci_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

baseline model

In [36]:
!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git && pip install -r stepik-dl-nlp/requirements.txt
import sys; sys.path.append('./stepik-dl-nlp')

fatal: destination path 'stepik-dl-nlp' already exists and is not an empty directory.


In [37]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import collections

import torch
from torch import nn
from torch.nn import functional as f

import dlnlputils
from dlnlputils.data import tokenize_text_simple_regex, tokenize_corpus, build_vocabulary,vectorize_texts,SparseFeaturesDataset
from dlnlputils.pipeline import train_eval_loop,predict_with_model,init_random_seed

init_random_seed()

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
from google.colab import files
files.upload()

Saving nlp.xlsx to nlp (2).xlsx


{'nlp.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee\x9dh^\x01\x00\x00\x90\x04\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [40]:
import pandas as pd

In [41]:
data = pd.read_excel('nlp.xlsx')

In [42]:
data.head()

,Что повлияло на Вашу оценку?,Что повлияло на Вашу оценку?_2,правильный ответ,оценка
0,"нравятся акции, рядом с домом","нравятся акции, рядом с домом",Акции,Положительно
1,"Красиво,чисто","Красиво,чисто",Комфорт магазина,Положительно
2,"Акции хорошие, ассортимент","Акции хорошие, ассортимент",Акции,Положительно
3,Высокие цены,Высокие цены,Цены,Отрицательно
4,Очень дорого,Очень дорого,Цены,Отрицательно


In [43]:
data = data.drop('Что повлияло на Вашу оценку?_2',axis=1)

In [44]:
import re 

In [45]:
#применим функцию к нижниму регистру
def capitalizer(string: str) -> str:
    return string.lower()
# Применить функцию
data['Что повлияло на Вашу оценку?'] = [capitalizer(string) for string in data['Что повлияло на Вашу оценку?']]

In [46]:
def change_coma(x):
  return x.replace(',',' ')
data['Что повлияло на Вашу оценку?'] = [change_coma(string) for string in data['Что повлияло на Вашу оценку?']]

In [47]:
#удаление ссылок и разных символов
def remove_urls(x):
    return re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , x)

In [48]:
#удаление почтывых ссылок 
def remove_emails(x):
     return re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", x)

In [49]:
def remove_rt(x):
    return re.sub(r'\brt\b', '', x).strip()

In [50]:
def remove_special_chars(x):
    x = re.sub(r'[^\w ]+', "", x)
    x = ' '.join(x.split())
    return x

In [51]:
def remove_numbers(s, keep=None):
    if keep:
        keep = set(str(x) for x in keep)
        return re.sub(r'\b\d+\b', lambda m: m.group() if m.group() in keep else '', s)
    else:
        # Shortcircuit the use of a set if there is no sequence to keep
        return re.sub(r'\b\d+\b', '', s)

In [52]:
data['text_clean'] = data['Что повлияло на Вашу оценку?'].apply(lambda x: remove_emails(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_urls(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_rt(x))
data['text_clean'] = data['text_clean'].apply(lambda x: remove_special_chars(x))
data['text_clean'] = data['text_clean'].apply(lambda s: remove_numbers(s))

In [53]:
data.head()

,Что повлияло на Вашу оценку?,правильный ответ,оценка,text_clean
0,нравятся акции рядом с домом,Акции,Положительно,нравятся акции рядом с домом
1,красиво чисто,Комфорт магазина,Положительно,красиво чисто
2,акции хорошие ассортимент,Акции,Положительно,акции хорошие ассортимент
3,высокие цены,Цены,Отрицательно,высокие цены
4,очень дорого,Цены,Отрицательно,очень дорого


In [54]:
#применим функцию к нижниму регистру
def capitalizer(string: str) -> str:
    return string.lower()
# Применить функцию
data['answer'] = [capitalizer(string) for string in data['правильный ответ']]

In [55]:
#применим функцию к нижниму регистру
def capitalizer(string: str) -> str:
    return string.lower()
# Применить функцию
data['rate'] = [capitalizer(string) for string in data['оценка']]

In [56]:
data.head()

,Что повлияло на Вашу оценку?,правильный ответ,оценка,text_clean,answer,rate
0,нравятся акции рядом с домом,Акции,Положительно,нравятся акции рядом с домом,акции,положительно
1,красиво чисто,Комфорт магазина,Положительно,красиво чисто,комфорт магазина,положительно
2,акции хорошие ассортимент,Акции,Положительно,акции хорошие ассортимент,акции,положительно
3,высокие цены,Цены,Отрицательно,высокие цены,цены,отрицательно
4,очень дорого,Цены,Отрицательно,очень дорого,цены,отрицательно


In [57]:
from sklearn.preprocessing import LabelEncoder

In [58]:
encoder = LabelEncoder()

In [59]:
data['answer'] = encoder.fit_transform(data['answer'])

In [60]:
data['rate'] = data['rate'].map({'положительно':0,'отрицательно':1})

In [61]:
X = data['text_clean']
y = data['answer']

In [62]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=17)

In [63]:
train_tokenized = tokenize_corpus(X_train)
test_tokenized = tokenize_corpus(X_test)

print(' '.join(train_tokenized[0]))

чисто хороший хлеб


In [64]:
MAX_DF = 0.8 #токены которые встречаются более чем в 80% документа нас не интересуют, потому что они слишком частотные
MIN_COUNT = 3 #слова встретившиеся менее 5 раз нас не интересуют
vocabulary,word_doc_freq = build_vocabulary(train_tokenized,max_doc_freq=MAX_DF,min_count=MIN_COUNT)
UNIQUE_WORDS_N = len(vocabulary)
print('Количество уникальных токенов', UNIQUE_WORDS_N)
print(list(vocabulary.items())[:10])

Количество уникальных токенов 303
[('ассортимент', 0), ('цены', 1), ('хороший', 2), ('нравится', 3), ('рядом', 4), ('акции', 5), ('выбор', 6), ('магазин', 7), ('хорошие', 8), ('дорого', 9)]


In [65]:
VECTORIZATION_MODE = 'tfidf'
train_vectors = vectorize_texts(train_tokenized,vocabulary,word_doc_freq,mode=VECTORIZATION_MODE)
test_vectors = vectorize_texts(test_tokenized,vocabulary,word_doc_freq,mode=VECTORIZATION_MODE)

print('the shape of matrix of the train values', train_vectors.shape)
print('the shape of matrix of the test values', test_vectors.shape)
print()
print('count of the none zero elem of matrix', train_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(train_vectors.nnz * 100 / (train_vectors.shape[0] * train_vectors.shape[1])))
print()
print('Количество ненулевых элементов в тестовой выборке', test_vectors.nnz)
print('Процент заполненности матрицы признаков {:.2f}%'.format(test_vectors.nnz * 100 / (test_vectors.shape[0] * test_vectors.shape[1])))

the shape of matrix of the train values (1784, 303)
the shape of matrix of the test values (447, 303)

count of the none zero elem of matrix 5521
Процент заполненности матрицы признаков 1.02%

Количество ненулевых элементов в тестовой выборке 1279
Процент заполненности матрицы признаков 0.94%


In [67]:
UNIQUE_LABELS_N = len(set(data['answer']))
print('Количество уникальных меток', UNIQUE_LABELS_N)

Количество уникальных меток 11


In [75]:
y_test = y_test.values
y_train = y_train.values

In [76]:
train_dataset = SparseFeaturesDataset(train_vectors,y_train)
test_dataset = SparseFeaturesDataset(test_vectors,y_test)

In [77]:
model = nn.Linear(UNIQUE_WORDS_N, UNIQUE_LABELS_N)

scheduler = lambda optim: \
    torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=5, factor=0.5, verbose=True)

best_val_loss, best_model = train_eval_loop(model=model,
                                            train_dataset=train_dataset,
                                            val_dataset=test_dataset,
                                            criterion=f.cross_entropy,
                                            lr=1e-1,
                                            epoch_n=200,
                                            batch_size=32,
                                            l2_reg_alpha=0,
                                            lr_scheduler_ctor=scheduler)

Эпоха 0
Эпоха: 56 итераций, 0.43 сек
Среднее значение функции потерь на обучении 2.170293073569025
Среднее значение функции потерь на валидации 2.0676349997520447
Новая лучшая модель!

Эпоха 1
Эпоха: 56 итераций, 0.28 сек
Среднее значение функции потерь на обучении 1.8851622364350729
Среднее значение функции потерь на валидации 1.9100660596575056
Новая лучшая модель!

Эпоха 2
Эпоха: 56 итераций, 0.28 сек
Среднее значение функции потерь на обучении 1.6872376160962241
Среднее значение функции потерь на валидации 1.8119818823678153
Новая лучшая модель!

Эпоха 3
Эпоха: 56 итераций, 0.26 сек
Среднее значение функции потерь на обучении 1.5264963997261864
Среднее значение функции потерь на валидации 1.7294946312904358
Новая лучшая модель!

Эпоха 4
Эпоха: 56 итераций, 0.25 сек
Среднее значение функции потерь на обучении 1.4125683243785585
Среднее значение функции потерь на валидации 1.67392098903656
Новая лучшая модель!

Эпоха 5
Эпоха: 56 итераций, 0.26 сек
Среднее значение функции потерь на о

In [78]:
train_pred = predict_with_model(best_model,train_dataset)

train_loss = f.cross_entropy(torch.from_numpy(train_pred),
                             torch.from_numpy(y_train).long())

print('Среднее значение функции потерь на обучении',float(train_loss))
print('Доля верных ответов',accuracy_score(y_train,train_pred.argmax(-1)))
print()

test_pred = predict_with_model(best_model,test_dataset)

test_loss = f.cross_entropy(torch.from_numpy(test_pred),
                            torch.from_numpy(y_test).long())

print('Среднее значение функции потерь на валидации', float(test_loss))
print('Доля верных ответов', accuracy_score(y_test, test_pred.argmax(-1)))

100%|██████████| 56/55.75 [00:00<00:00, 238.64it/s]


Среднее значение функции потерь на обучении 0.717770516872406
Доля верных ответов 0.7920403587443946



100%|██████████| 14/13.96875 [00:00<00:00, 202.30it/s]

Среднее значение функции потерь на валидации 1.4887539148330688
Доля верных ответов 0.5771812080536913


In [79]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

sklearn_pipeline = Pipeline((('vect', TfidfVectorizer(tokenizer=tokenize_text_simple_regex,
                                                      max_df=MAX_DF,
                                                      min_df=MIN_COUNT)),
                             ('cls', LogisticRegression())))
sklearn_pipeline.fit(X_train, y_train);

In [81]:
sklearn_train_pred = sklearn_pipeline.predict_proba(X_train)
sklearn_train_loss = f.cross_entropy(torch.from_numpy(sklearn_train_pred),
                                                 torch.from_numpy(y_train))
print('Среднее значение функции потерь на обучении', float(sklearn_train_loss))
print('Доля верных ответов', accuracy_score(y_train, sklearn_train_pred.argmax(-1)))
print()

sklearn_test_pred = sklearn_pipeline.predict_proba(X_test)
sklearn_test_loss = f.cross_entropy(torch.from_numpy(sklearn_test_pred),
                                                torch.from_numpy(y_test))
print('Среднее значение функции потерь на валидации', float(sklearn_test_loss))
print('Доля верных ответов', accuracy_score(y_test, sklearn_test_pred.argmax(-1)))

Среднее значение функции потерь на обучении 1.9798796558666256
Доля верных ответов 0.7948430493273543

Среднее значение функции потерь на валидации 2.038980614511817
Доля верных ответов 0.6823266219239373
